Мета:

Розробити та налаштувати GAN для задачі відновлення зображень (наприклад, збільшення роздільної здатності, відновлення пошкоджених зображень), а також оцінити ефективність отриманої моделі.

Кроки для виконання завдання:

1. Встановлення середовища:
  * Встановіть необхідні бібліотеки: TensorFlow або PyTorch, numpy, matplotlib, OpenCV (для обробки зображень).
  * Підготуйте обчислювальне середовище з GPU для швидшого навчання, якщо можливо.
2. Вибір та підготовка набору даних:
  * Виберіть набір даних для відновлення зображень, наприклад, CelebA (для облич) або інший датасет зображень.
  * Підготуйте навчальні приклади: створіть зменшені або пошкоджені версії зображень для тренування GAN (наприклад, зменшення роздільної здатності, розмиття або додавання шуму).
3. Створення архітектури GAN для відновлення зображень:
  * Генератор:
    * Побудуйте генератор, що прийматиме зображення з низькою роздільною здатністю (або пошкоджене) та відновлюватиме його до високоякісної версії.
    * Використайте згорткові шари (Conv2DTranspose) та шари Batch Normalization для кращої стабільності.
    * Додайте функцію активації ReLU або LeakyReLU в прихованих шарах, і tanh для вихідного шару.
  * Дискримінатор:
    * Побудуйте дискримінатор для розпізнавання відновлених (штучних) зображень від реальних.
    * Використайте згорткові шари з функціями активації LeakyReLU та нормалізацією пакетів (Batch Normalization).
    * Завершіть дискримінатор функцією активації sigmoid для класифікації (реальне/згенероване).
4. Налаштування функцій втрат та оптимізаторів:
  * Використайте binary cross-entropy як функцію втрат для дискримінатора.
  * Для генератора використайте mean squared error або L1 втрати для відновлення зображення та помірні adversarial loss (для конкуренції з дискримінатором).
  * Налаштуйте оптимізатори (наприклад, Adam) з відповідними коефіцієнтами навчання (зазвичай, менший коефіцієнт для дискримінатора).
5. Навчання GAN:
  * Крок 1: Навчіть дискримінатор окремо, використовуючи реальні та згенеровані зображення.
  * Крок 2: Навчіть генератор для покращення якості зображень і обману дискримінатора.
  * Повторюйте цей процес навчання протягом декількох епох, поступово збільшуючи якість згенерованих зображень.
6. Оцінка ефективності:
  * Використайте метрики, такі як PSNR (Peak Signal-to-Noise Ratio) та SSIM (Structural Similarity Index) для оцінки якості відновлених зображень.
  * Проведіть візуальне порівняння між відновленими зображеннями та оригіналами для оцінки точності відновлення.

Мінімальні вимоги:

* Реалізувати GAN для відновлення зображень та провести базову оцінку ефективності за допомогою метрик PSNR та SSIM.

Додаткові рекомендації:

* Спробуйте інші архітектури GAN, такі як SRGAN (Super-Resolution GAN) для покращення якості відновлення зображень.

Формат виконання: 

.ipynb блокнот з кодом та візуалізацією, або ж код в .py з прикріпленими зображеннями результатів на гітхабі.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.utils as vutils
from tqdm import tqdm  # Import tqdm for progress bar

# Define the Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),  # (B, 3, 64, 64) -> (B, 64, 32, 32)
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1),  # (B, 64, 32, 32) -> (B, 128, 16, 16)
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1),  # (B, 128, 16, 16) -> (B, 256, 8, 8)
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1),  # (B, 256, 8, 8) -> (B, 512, 4, 4)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),  # (B, 512, 4, 4) -> (B, 256, 8, 8)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),  # (B, 256, 8, 8) -> (B, 128, 16, 16)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # (B, 128, 16, 16) -> (B, 64, 32, 32)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1),  # (B, 64, 32, 32) -> (B, 3, 64, 64)
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),  # (B, 3, 64, 64) -> (B, 64, 32, 32)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1),  # (B, 64, 32, 32) -> (B, 128, 16, 16)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1),  # (B, 128, 16, 16) -> (B, 256, 8, 8)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1),  # (B, 256, 8, 8) -> (B, 512, 4, 4)
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0),  # (B, 512, 4, 4) -> (B, 1, 1, 1)
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1).squeeze(1)

# Hyperparameters
batch_size = 64
lr = 0.0002
image_size = 64
num_epochs = 5

# Data preparation
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = datasets.CelebA(root='./data', split='train', transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize models, optimizers, and loss function
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
netG = Generator().to(device)
netD = Discriminator().to(device)
criterion = nn.BCELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

# Training Loop with tqdm
for epoch in range(num_epochs):
    loop = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for i, (data, _) in loop:
        # Prepare real and damaged images
        real_images = data.to(device)
        batch_size = real_images.size(0)
        mask = torch.ones_like(real_images)
        mask[:, :, 16:48, 16:48] = 0  # Random block mask
        damaged_images = real_images * mask

        # Labels
        real_labels = torch.ones(batch_size, device=device)
        fake_labels = torch.zeros(batch_size, device=device)

        # Train Discriminator
        netD.zero_grad()
        output_real = netD(real_images)
        loss_real = criterion(output_real, real_labels)

        fake_images = netG(damaged_images)
        output_fake = netD(fake_images.detach())
        loss_fake = criterion(output_fake, fake_labels)

        lossD = loss_real + loss_fake
        lossD.backward()
        optimizerD.step()

        # Train Generator
        netG.zero_grad()
        output_fake = netD(fake_images)
        lossG = criterion(output_fake, real_labels)
        lossG.backward()
        optimizerG.step()

        # Update tqdm progress bar
        loop.set_postfix(lossD=lossD.item(), lossG=lossG.item())

    # Save images and models after each epoch
    vutils.save_image(fake_images, f"output/fake_epoch_{epoch}.png", normalize=True)
    torch.save(netG.state_dict(), "generator.pth")
    torch.save(netD.state_dict(), "discriminator.pth")

Files already downloaded and verified


Epoch [1/5]:  69%|██████▉   | 1761/2544 [7:25:51<3:18:14, 15.19s/it, lossD=1.01, lossG=3]     


KeyboardInterrupt: 